# <center> **Лекция 6:** Компьютерное зрение. Задача Landmark Detection. </center>
----
<br>
</br>

<center> <img src=./../src/imgs/landmark_intro_img.png> </center>

<br>
</br>

## __План на сегодня:__

----
### 1. _Задача Landmark Detection. Human Pose Estimation. 2D и 3D Landmarks._
### 2. _Facial Landmarks Detection._

# Часть 1:

____

# _Задача Landmark Detection. Human Pose Estimation. 2D и 3D Landmarks._

----

Landmarks (точки-ориентиры) - это ключевые точки, которые привязаны к определенным частям объекта (человека, лица, зданий, скульптур и т.д.).

Начнем разбор задачи Landmark Detection с задачи __Human Pose Estimation__. Задача __Human Pose Estimation__ определяется как проблема локализации сочленений, суставов (joints), которые также известны как ключевые точки - локти, запястья и т.д., в изображениях или видеопотоке. Также задачу можно свести к поиску конкретной позы в пространстве всех поз (то есть классификации). Задачу можно решать в разной геометрии:

* __2D Pose Estimation__ - то есть 2D пространство, где у нас есть набор координат (x,y) для входного изображения.
* __3D Pose Estimation__ - то есть 3D пространство, где у нас есть набор (x,y,z) координат для входного изображения RGB.



Где все это может применяться:

1. Action Recognition.
2. Animation - Motion Capture.
3. Sport - Action Tracking, Situational Judgement (Football - VAR, Basketball etc).
4. 2D / 3D Reconstruction (Archeology, Science, Cinema).


[Пример работы 1.](https://www.youtube.com/watch?v=RMgrAxds3DU)
[Пример работы 2.](https://www.youtube.com/watch?v=Dhkd_bAwwMc)

____

# _2D Pose Estimation._

----

Начнем разбор задачи с 2D Pose Estimation, когда нам необходимо проставить (x,y) координаты для найденного объекта и определить позу. Но прежде чем коснуться этой задачи поговорим как всегда о том, как оценивать качество реализованного решения для таких задач.

<br>
</br>

__Метрики__:

1. __PCP@0.5__ (_Percentage of Correct Parts_) - показывает какой % картинок с правильно обнаржуенными джойнтами (конечностями, соединениями). Считается отдельно для каждого джойнта (конечности, соединения), затем усредняется. __Джойнт обнаружен__ если расстояние между двумя предполагаемыми местоположениями суставов и истинными местоположениями суставов конечностей составляет не более половины длины (0.5) джойнта (конечности, соединения). Суть:
    * Она штрафует более короткие конечности в большей степени, поскольку у более коротких конечностей пороги меньше.
    * Чем выше показатель/метрика PCP тем лучше модель.


2. __PCK__ (_Percentage of Correct Key-Points) - предсказанный джойнт (сустав, конечность, соединение) считается правильным, если расстояние между прогнозируемым и истинным местоположением джойнта находится в пределах определенного порога (задается на джойнт - например для головы, руки, ноги и тд). Порог может быть следующим:
    * __PCKh@0.5__ - расстояние между предсказанной точкой и GroundTruth < 50% (head ling)
    * __PCK@0.2__ - расстояние между предсказанной точкой и GroundTruth < 0.2  (torso diameter)
    * __PCK__ используется как для 2D так и для 3D (PCK3D)
    * Чем выше тем лучше модель.



<br>
</br>

Раньше задачу решали классическим ML, но из-за высокого потребления ресурсов и низкого качества, практически нигде не использовалось. Пока не научились решать задачу в контексте глубокого обучения.


<br>
</br>

----
### __Deep Pose: Human Pose Estimation via Deep Neural Networks (CVPR'14)__:
----
<br>
</br>

<center> <img src=./../src/imgs/deepposeestimation.png> </center>

<br>
</br>

Самое просто что можно сделать для решения такой задачи - у нас есть картинка на входе, нам нужно выдать например 16 локализаций точек. Что будем делать и как решать - взял просто в качестве экстрактора признаков CNN и решить задачу регрессии для (x,y) джойнтов и обучить алгоритм, таким образом на выходе мы получим k-точек. Звучит просто.

[В изначальном подходе CVPR'14:](https://arxiv.org/pdf/1312.4659.pdf)

__Первый шаг (Initial Stage)__:
1. В качестве CNN бекенда выступала AlexNet (7 слоев) + 1 дополнительный регрессионный слой, который дает на выходе __2*k__ совместных джойнтов, где k - количество точек необходимы для локализации.
2. Модель обучалась используя L2 loss функцию для регрессионной компоненты.

__Второй шаг (Stage S)__:
1. После того как предсказали (x,y) точку для джойнта, эта точка отдается дальше для решения задачи Refinement то есть определения области "взаимодействия" данной точки (задача схожа с Bounding Box регрессией).

<br>
</br>

Но такая архитектура плохо справлялась с отдельными джойнтами и вовсе не детектила их. Проблема - отсутствие глобального контекста и невозможность хранить весь объем информации о пространстве признаков всего изображения (особенно для регрессии когда нам надо предсказывать огромного количество данных).


Как попытались решить проблему эффективности дальше:

<br>
</br>

----
### [__Efficient Object Localization Using Convolutional Neural Networks (CVPR'15)__](https://arxiv.org/pdf/1411.4280.pdf):
----
<br>
</br>

<center> <img src=./../src/imgs/heatmapcvpr.png> </center>

<br>
</br>

Предложили другой подход вместо регрессии на 2*k точек. Попытались внедрить глобальный контекст. Реализовали подход через heatmaps, где каждая подсвеченная область на карте изображения соответствует одной лендмарке. Эти heatmaps получаются в результате предсказания нейронной сети. Дальше мы можем их смержить в одно предсказание, которое будет представлять позу объекта на изображении.

<br>
</br>

Кратко как вся эта архитектура обучается:

<br>
</br>

<center> <img src=./../src/imgs/cvpr15.png> </center>

<br>
</br>

Выходом модели будет K-тепловых карт размера $W_0 \times H_0, {H_1, H_2, ..., H_k}$ где каждая тепловая карта $H_k$ соответствует локализации k-го ландмарка с определенной степенью уверенности.

<br>
</br>

----
### [__Convolutional Pose Machines (CVPR'16)__](https://arxiv.org/pdf/1602.00134.pdf):
----
<br>
</br>

<center> <img src=./../src/imgs/convolutional_pose_machines_1.png> </center>

<br>
</br>

<br>
</br>

<center> <img src=./../src/imgs/convolutional_pose_machines_2.png> </center>

<br>
</br>

* Мультишаговая архитектура (несколько задач).
* Используем промежуточное обучение с учителем.
* Работает с пространственной информацией (spatial information) используя большие группы рецептивных полей (receptive fields).

<br>
</br>

Кратко как вся эта архитектура обучается:

<br>
</br>

<center> <img src=./../src/imgs/convolutional_pose_machines_arrch.png> </center>

<br>
</br>

Как обстоят дела с SOTA бенчмарками:

<br>
</br>

<center> <img src=./../src/imgs/SOTA_pose.png> </center>

<br>
</br>

На каком датасете все учится и проверяется (Human Pose Estimation задача) - [MPII Human Pose Dataset](http://human-pose.mpi-inf.mpg.de/)

<br>
</br>

----
### [__Stacked Hourglass Networks for Human Pose Estimation__](https://arxiv.org/pdf/1603.06937.pdf):
----
<br>
</br>

<center> <img src=./../src/imgs/hourglass_networks.png> </center>

<br>
</br>

Альтернативная идея работы с глобальным контекстом. Логика - сжать представление до самого маленького пространства, которое даст максимальную информативность для локализации точки и добавить скип коннекшн чтобы не потерять основную часть информации. В конце получаем набор хитмапов, которые дают наиболее локализованные и точные точки для детекта позы.

<br>
</br>

Идея архитектуры:
* Используем скип коннекшены для того чтобы сохранять пространственную информацию
* Используем промежуточное обучение с учителем
* Вся сеть набор сверток с остаточными модулями (residual modules)
* Можем забирать как локальный так и глобальный контекст

<br>
</br>

Как выглядит архитектура такого блока:

<br>
</br>

<center> <img src=./../src/imgs/hourglass_architecture.png> </center>

<br>
</br>

Ссылки на решения:

* [Реализация для TensorFlow](https://github.com/ethanyanjiali/deep-vision/tree/master/Hourglass/tensorflow)
* [Библиотека MMPose которая содержит архитектуру для PyTorch и другие модели](https://github.com/open-mmlab/mmpose)

Минусы решения:
1. Проблема с детектом для нескольких объектов в кадре.

<br>
</br>

----
### [__Deep High-Resolution Representation Learning for Human Pose Estimation (HRNet) (CVPR’19)__](https://arxiv.org/pdf/1902.09212.pdf):
----

<br>
</br>

<center> <img src=./../src/imgs/HRNET.png> </center>

<br>
</br>

* Лучше по метрикам чем Hourglass
* Используем деконволюции вместо апсэмплинга (transopes convolutions)
* Такая архитектура начинается с нейронной сети для карт признаков выского разрешения (первый этап), добавляем модули которые отвечают за генерацию карт признаков для низкого разрешения (второй этап) и соединяем это все мульти-признаковую параллельную нейронную сеть.
* Задача генерации тепловых карт точек (landmarks) решается путем регрессии используя MSE в качестве функции потерь.

[Реализация](https://github.com/leoxiaobin/deep-high-resolution-net.pytorch)


----
### [__ViTPose: Simple Vision Transformer Baselines for Human Pose Estimation (CVPR’21)__](https://arxiv.org/pdf/2204.12484.pdf):
----

<br>
</br>

<center> <img src=./../src/imgs/vitpose.png> </center>

<br>
</br>


### Реализация и использование технологии для детекта позы:

### Модули и библиотеки:

Нам понадобится библиотека [MediaPipe](https://google.github.io/mediapipe/) которая содержит огромное количество кросс-платформерного функционала а также реализаций кастомных моделей. В настоящее время используется одна из SOTA моделей [BlazeFace](https://arxiv.org/pdf/1907.05047.pdf) автором архитектуры которой является [Андрей Вакунов](https://www.linkedin.com/in/andrey-vakunov-25194022/) (альма-маттер - Гродненский Государственный Университет Янки Купалы).

<br>
</br>

```
>>> pip install mediapipe opencv-python
```

In [3]:
import cv2
import mediapipe as mp
import numpy as np

# Функционал для рендеринга позы и точек landmarks
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

1. Раз у нас есть вебкамера, то вместо картинок давайте поиграемся с live-видео и посмотрим как это выглядит в живую:

In [2]:
# Захват видеопотока вебкамерой
cap = cv2.VideoCapture(0) # аргумент - номер девайса вебкамеры

while cap.isOpened():
    ret, frame = cap.read() # читаем фрейм (кусочек кадра с видеопотока)
    cv2.imshow('Фрейм: ', frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

2. Реализуем детекции с использованием уже готовой модели для Pose Estimation:

In [4]:
import cv2
cap = cv2.VideoCapture(0)
## Инициализируем экземпляр mediapipe который вызовет модель BlazeFace для задачи Pose Estimation
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        # Перевод картинги из BGR > RGB не забывайте всегда переводить цветовые каналы
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Детект позы:
        results = pose.process(image)
        # Обратный перевод в BGR для визуализации
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Рендеринг направлений
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [5]:
results.pose_landmarks
mp_pose.POSE_CONNECTIONS

frozenset({(0, 1),
           (0, 4),
           (1, 2),
           (2, 3),
           (3, 7),
           (4, 5),
           (5, 6),
           (6, 8),
           (9, 10),
           (11, 12),
           (11, 13),
           (11, 23),
           (12, 14),
           (12, 24),
           (13, 15),
           (14, 16),
           (15, 17),
           (15, 19),
           (15, 21),
           (16, 18),
           (16, 20),
           (16, 22),
           (17, 19),
           (18, 20),
           (23, 24),
           (23, 25),
           (24, 26),
           (25, 27),
           (26, 28),
           (27, 29),
           (27, 31),
           (28, 30),
           (28, 32),
           (29, 31),
           (30, 32)})

3. Реализуем определение джойнтов (joints):

Откуда можно взять лейблы точек [https://google.github.io/mediapipe/solutions/pose.html](https://google.github.io/mediapipe/solutions/pose.html)

In [6]:
cap = cv2.VideoCapture(0)
## Инициализируем экземпляр mediapipe который вызовет модель BlazeFace для задачи Pose Estimation
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        # Перевод картинги из BGR > RGB не забывайте всегда переводить цветовые каналы
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Детект позы:
        results = pose.process(image)
        # Обратный перевод в BGR для визуализации
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Выявим лендмарки:
        try:
            landmarks = results.pose_landmarks.landmark # лежит все тут
            # print(landmarks)
        except:
            pass

        # Рендеринг направлений
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [7]:
len(landmarks)

33

In [8]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [9]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility

0.9994725584983826

In [10]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.91986144
y: 0.94500583
z: -0.44118562
visibility: 0.7961808

In [11]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 1.0874245
y: 1.2875625
z: -0.55753887
visibility: 0.6846311

4. Посчитаем углы между плечом и локтем для того чтобы добавить функционал для расчета подходов для поднимания гантелей :)):

In [12]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle = 360-angle

    return angle

In [13]:
# Доастанем координаты joint'ов для того чтобы реализовать функционал
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [14]:
shoulder, elbow, wrist

([0.7339316606521606, 0.646515965461731],
 [0.9198614358901978, 0.9450058341026306],
 [1.0874245166778564, 1.2875624895095825])

In [15]:
calculate_angle(shoulder, elbow, wrist)

174.14693135746353

In [16]:
tuple(np.multiply(elbow, [640, 480]).astype(int))

(588, 453)

Соединим и псмотрим что получилось:

In [17]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            angle = calculate_angle(shoulder, elbow, wrist)

            cv2.putText(image, str(angle),
                           tuple(np.multiply(elbow, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )

        except:
            pass

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

5. Реализуем счетчик подходов для поднимания гантели на бицепс :D

In [ ]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            angle = calculate_angle(shoulder, elbow, wrist)

            cv2.putText(image, str(angle),
                           tuple(np.multiply(elbow, [640, 480]).astype(int)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            # Посчитаем подходы :
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)

        except:
            pass

        # Создадим бокс который будет отображать результат :)
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)

        cv2.putText(image, 'REPS', (15,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter),
                    (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        cv2.putText(image, 'STAGE', (65,12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage,
                    (60,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)


        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

### Практика: Добавьте функционал еще и на правую руку и выведите результат отдельно для левой и для правой руки:

```
--------
Левая:  0
Правая: 1
--------
````

____

# _3D Pose Estimation._

----

Почему 3D сложнее чем 2D :
1. Огромное количество вариаций точек
2. Недостаток 3D датасетов реальных данных
3. Сложность определения джойнтов и ландмарков


__Какие метрики используются:__

__MPJPE (Mean Per Joint Position Error) -- это L2 расстояние, усредненное по всем лэндмаркам__


Что имеем из исследований на данный момент:

----
### [__Unsupervised Geometry-Aware Representation for 3D Human Pose Estimation  (ECCV'19)__](https://arxiv.org/pdf/1804.01110.pdf):
----

<br>
</br>

<center> <img src=./../src/imgs/3dPose.png> </center>

<br>
</br>

В чем идея:
1. Используем автоэнкодер для того чтобы научится 3D представлениям
2. Используем обучение с учителем для маппинга таких репрезентаций с позами в задаче 3N регрессии где N - количество джойнтов.

Во время инференса, расчитывается все латентные представления изображения и отдаем в качестве инпута нейронной сетке для того чтобы рассчитать позу.


Как происходит обучение репрезентации:

<br>
</br>

<center> <img src=./../src/imgs/representation_learning_pose.png> </center>

<br>
</br>


[Реализация](https://github.com/hrhodin/UnsupervisedGeometryAwareRepresentationLearning)

<br>
</br>

----
### [__3D Human Pose Machines with Self-supervised Learning__](https://arxiv.org/pdf/1901.03798.pdf):
----

<br>
</br>

<center> <img src=./../src/imgs/3dposemachinessp.png> </center>

<br>
</br>

Идея:
1. Можем извлекать репрезентативную информацию как о 2D так и о 3D/
2. Почему бы не добавить информацию о 2D пространстве в генерацию 3D поз чтобы решить еще задачу максимального соответствия позы в любом R (ракурс поворота.)
3. Закодируем все признаки и вытащим их через LSTM
4. Декодируем и получим точки для позы которые соответствуют ground truth Объекту.

<br>
</br>

<center> <img src=./../src/imgs/3dhumanposemachinesarch.png> </center>

<br>
</br>



# Часть 2:

____

# _Задача Facial Landmarks Detection._

----

На чем обучают:
1. Своя разметка
2. [WFLW, 98 landmarks](https://wywu.github.io/projects/LAB/WFLW.html)
3. [DLIB, 68 and 5 landmarks](https://github.com/davisking/dlib-models)
4. [3D Facial Landmarks](https://github.com/1adrianb/face-alignment)

<br>
</br>

Применение:
1. Перенос стиля и лица (маски, дипфейки, замена лица и тд).
2. Трекинг эмоций, реакций.
3. Поиск лиц по базе, по скрытым (закрытым) фотография - деаномизация.
4. Трекинг внимания (Face ID и тд).
5. Выравнивание лица по бенчмарку.
6. Трекинг только лиц в видеопотоке.
7. Медицина - диагнозы, поиск аномалий, поиск болезней по данным снимкам и тд.
8. Биометрия.

[Реализация Face Mash](https://github.com/Yinghao-Li/3DMM-fitting)
[Реализация HypsterizerCats](https://github.com/kairess/cat_hipsterizer)

Решение задачи - такая же как у и задач пройденных выше, только для определенной области, где мы по максимуму извлекаем признаки. Самый простой подход - регрессия на 2N значений (где N - количество лэндмарков).

Что еще есть из уникального:

----
### [__PFLD: A Practical Facial Landmark Detector__](https://arxiv.org/pdf/1902.10859.pdf):
----

<br>
</br>

<center> <img src=./../src/imgs/pfld.png> </center>

<br>
</br>

Идея:
1. Вспомогательная сеть, которая использует общую архитектуру и прогнозирует повороты (углы рта, области лица и кривые лица и тд).
2. В качестве CNN бэкенда - MobileNet блочная CNN
3. L2 loss + геометрия пространства (данные из датасета) + аугментация картинок.


[Реализация](https://github.com/guoqiangqi/PFLD)

----
### [__Attention-Driven Cropping for Very High Resolution Facial Landmark Detection__](http://studios.disneyresearch.com/2020/06/16/attention-driven-cropping-for-very-high-resolution-facial-landmark-detection/):
----

<br>
</br>

<center> <img src=./../src/imgs/attention_based_arch.png> </center>

<br>
</br>

Идея:
1. Берем очень высокое разрешение.
2. Получаем хитмап при помощи hourglass архитектуру и используем в качестве бенчмарка.
3. Нарезаем на кусочки при помощи механизма внимания и отдаем все в параллельный hourglass для обучения нескольких паттернов лица.
4. Соединяем полученные лендмарки и сравниваем с бенчмарком.
5. Получаем лендмарки.


Интересные проекты:
1. [Learnable Triangulation of Human Pose](https://github.com/karfly/learnable-triangulation-pytorch)
2. [Face Alignment using a Deeply-initialized Coarse-to-fine Ensemble of Regression Trees](https://github.com/bobetocalo/bobetocalo_eccv18)
3. [SPIGA: Shape Preserving Facial Landmarks with Graph Attention Networks.](https://github.com/andresprados/spiga)
4. [Cascade of Encoder-Decoder CNNs with Learned Coordinates Regressor for Robust Facial Landmarks Detection](https://github.com/bobetocalo/bobetocalo_prl19)

In [18]:
import mediapipe as mp
import cv2
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

Добавим в код который мы написали выше функционал для детекта Facial Landmarks:

In [20]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Задетектим facial landmards
        results = holistic.process(image)
        # print(results.face_landmarks)
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks,mp_holistic.FACEMESH_CONTOURS)
        # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        # Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        cv2.imshow('Raw Webcam Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Немного стилизуем нашу маску:

In [22]:
mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )

        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

        cv2.imshow('Raw Webcam Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()